In [3]:
import comm as cmm
import codecs

def fmtStrQuote(v):
    return str(v) if(type(v)==int) else f"'{str(v)}'"

def fmtOrderBy(v):
	return "" if(v=="") else f"""\t\t\t\tCASE WHEN @SortColumn = '{v}' AND @SortOrder = 'ASC' THEN f.CreateDT END ASC,\n\t\t\t\tCASE WHEN @SortColumn = '{v}' AND @SortOrder = 'DESC' THEN f.CreateDT END DESC,"""

def fmtQueryWhere(v):
    return "" if(v=="") else f""" f.{v} LIKE CASE WHEN @{v} = '' THEN f.{v} ELSE '%' + @{v} + '%' END\n\t\t\t\tAND"""

def fmtUpdateSet(v):
    return "" if(v=="") else f",{v} = @{v}"

def fmtInsertCols(v):
    return "" if(v=="") else f",{v}"

def fmtInsertVals(v):
    return "" if(v=="") else f",@{v}"

def GenCreateTable(df):
    tbNames = df["TableName"].unique()
    for tb in tbNames:
        colsStr, ColExtFormats="",""
        dfs = df[df["TableName"]==tb]
      
        for e in dfs["colComp"]:
            colsStr += e +"\n"

        for e in dfs["colExt"]:
            ColExtFormats += e +"\n"

        tmpFile = codecs.open("template/SQL/createTable.sql","r").read().format(tb=tb, colsStr=colsStr, ColExtFormats=ColExtFormats)
        f = codecs.open(f"tables/tb{tb}.sql", "w","utf-8")
        f.write(tmpFile)
        f.close()


In [4]:
import pandas as pd
import numpy as np
import pyodbc
import comm as cmm
import time
import codecs

df = pd.read_excel("tableSourceDef.xlsx", sheet_name="admin_columns",converters={'maxLen':str }).replace(np.nan, '')
df_sp = pd.read_excel("tableSourceDef.xlsx", sheet_name="SP_config",converters={'maxLen':str }).replace(np.nan, '')
df["colComp"] = "\t"+df["Column"].apply(cmm.fmtSquare) + (df["Type"]+df["maxLen"].apply(cmm.fmtParentheses)).apply(cmm.fmtSquare) + df["autoGen"]+ df["isNull"].apply(cmm.fmtNull)+","
df["colExt"] = "EXEC sys.sp_addextendedproperty @name=N'MS_Description', @value=N'" + df["ColDscr"]+ "' ,@level0type=N'SCHEMA',@level0name=N'" +df["Schema"]+"', @level1type=N'TABLE',@level1name=N'tb"+ df["TableName"]+"', @level2type=N'COLUMN',@level2name=N'"+df["Column"]+"'\nGO"
   
GenCreateTable(df)  ## 自動產生Tables

cnxn = pyodbc.connect(cmm.connMSSQL("newcti"))
query="select SN,tableName,col,fCol,cmmt_select,cmmt_SetValue,cmmt_Exec,fColOr  from [dbo].[myTbColumnDscr];"
dfc = pd.read_sql_query(query, cnxn).set_index("SN")
tbs = df_sp["TableName"].unique()


def GenSP(tb, df, dfc, keyCol, tmpFileTag):
    tbDscr = df_sp[df_sp["TableName"] == tb]["tbDscr"].values[0]

    df_IsExist = df[~(df[keyCol]=="")]                                      #主軸欄位:KeyColumn in Excel.
    dfc_IsExist = dfc[dfc["col"].isin(df_IsExist["Column"].tolist())]   #衍生出的另一個dataframe
    target = df_IsExist.merge(dfc_IsExist, left_on="Column", right_on="col")
    
    orderByList = df[df["orderBy"]=="V"]["Column"].tolist()             #找出orderBy欄位有哪些欄位
    df_OrderBy = df[df["Column"].isin(orderByList)]

    excludeForUpdate = ["SeqNo","Creator","CreateDT","UpdateDT","Updator"]  #設定那些欄位要被排除?
    excludeList = df[df["Column"].isin(excludeForUpdate)]["Column"].tolist()
    df_exclude = df[~df["Column"].isin(excludeList)]

    target["cmmt_Declares"] = ",@" + target["Column"] +" "+ target["Type"].str.upper() + target["maxLen"].apply(cmm.fmtParentheses)
    target["cmmt_InputParas"] = "@" + target["Column"] +"\t"+ target["Type"].str.upper() + target["maxLen"].apply(cmm.fmtParentheses) + " - " + target["ColDscr"]
    target["existSetVal"] =  "SET @" + target["Column"] + " = " + target["httpGet"].apply(fmtStrQuote)
    target["insertSetVal"] = "SET @" + target["Column"] + " = " + target["httpInsert"].apply(fmtStrQuote)
    target["updateSetVal"] = "SET @" + target["Column"] + " = " + target["httpUpdate"].apply(fmtStrQuote)
    target["querySetVal"] = "SET @" + target["Column"] + " = " + target["queryWhere"].apply(fmtStrQuote)

    pt_getSelectAll, pt_input, pt_Declare, pt_SetValue,pt_select, pt_fCol,pt_Exec,pt_fColOr,pt_orderBy,pt_queryWhere,pt_UpdateSet,pt_insertCols,pt_insertSetVal,pt_insertVals,pt_updateSetVal,pt_querySetVal = "","","","","","","","","","","","","","","",""

    target2 = target[target["col"] != "SeqNo"]
    
    pt_getSelectAll = "\n".join(df["cmmt_getSelectAll"].tolist())
    pt_select = "\n".join(target["cmmt_select"].tolist())
    pt_input = "\n\t".join(target["cmmt_InputParas"])
    pt_Declare = "\n\t".join(target["cmmt_Declares"])[1:]
    pt_fCol = "\n\t\t\t".join(target["fCol"])
    pt_Exec = "\n\t".join(target["cmmt_Exec"])[1:] 
    pt_fColOr = " OR\n\t\t\t\t".join(target2["fColOr"])
    pt_orderBy ="\n".join(df_OrderBy["Column"].apply(fmtOrderBy).tolist())[:-1]

    pt_queryWhere = "".join(df_exclude["Column"][df["Column"]!="IsEnable"].apply(fmtQueryWhere).tolist())[:-8] #queryWhere 屏除IsEnable，用來寫死控制。
    pt_UpdateSet = "\n\t\t\t".join(df_exclude["Column"].apply(fmtUpdateSet).tolist())[1:]
    pt_insertCols = "\n\t\t\t".join(df_exclude["Column"].apply(fmtInsertCols).tolist())[1:]
    pt_insertVals = "\n\t\t\t".join(df_exclude["Column"].apply(fmtInsertVals).tolist())[1:]

    pt_existSetValue = "\n\t".join(target["existSetVal"].tolist()) 
    pt_insertSetVal = "\n\t".join(target["insertSetVal"].tolist())
    pt_updateSetVal = "\n\t".join(target["updateSetVal"].tolist())
    pt_querySetVal = "\n\t".join(target["querySetVal"].tolist())

    tmpQueryFile = codecs.open(f"template/SQL/agdSP.{tmpFileTag}.sql",mode="r",encoding="utf-8",errors='strict', buffering=-1).read().format(tb=tb, tbDscr=tbDscr, pt_DateTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),pt_getSelectAll = pt_getSelectAll, pt_select = pt_select, pt_input = pt_input,pt_Declare= pt_Declare, pt_SetValue = pt_SetValue, pt_Exec = pt_Exec, pt_fCol=pt_fCol[1:], pt_fColOr = pt_fColOr, pt_orderBy = pt_orderBy, pt_queryWhere = pt_queryWhere, pt_UpdateSet = pt_UpdateSet, pt_insertCols = pt_insertCols, pt_insertVals = pt_insertVals, pt_insertSetVal = pt_insertSetVal, pt_updateSetVal = pt_updateSetVal, pt_querySetVal = pt_querySetVal, pt_existSetValue =pt_existSetValue)  #套表

    f = codecs.open(f"agdSP/usp{tb}{tmpFileTag}.sql", mode="w", encoding="utf-8", errors="strict")
    f.write(tmpQueryFile)
    f.close()

for tb in tbs:
    df["cmmt_getSelectAll"] = "**  " + df["Column"] +"\t - " + df["ColDscr"]
    _df, _dfc = df[df["TableName"] == tb], dfc[dfc["tableName"] == tb]

    GenSP(tb, _df, _dfc, "isExist","Exists")
    GenSP(tb, _df, _dfc, "isExist","Get")
    GenSP(tb, _df, _dfc, "isExist","Delete")
    GenSP(tb, _df, _dfc, "queryWhere","Query")
    GenSP(tb, _df, _dfc, "httpInsert","Insert")
    GenSP(tb, _df, _dfc, "httpUpdate","Update")
    

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
